In [8]:
import pandas as pd
import numpy as np

pd.set_option('mode.chained_assignment', None)

df = pd.read_csv(r"E:\Tung\Python\VcreamProject\cache\ALL.csv")

df.drop(['Unnamed: 0'], axis = 1, inplace=True)

In [22]:
vcb = df.loc[df['Ticker'] == "VCB"]
vcb 

,Ticker,Date,Open,High,Low,Close,Volume
2026933,VCB,2009-06-30,16.2420,16.2420,16.2420,16.2452,294070
2026934,VCB,2009-07-01,17.0541,17.0541,16.1067,16.3806,6248390
2026935,VCB,2009-07-02,16.1067,16.2420,15.5653,15.7037,1515670
2026936,VCB,2009-07-03,15.2946,15.4299,15.1592,15.1622,899720
2026937,VCB,2009-07-06,15.1592,15.8360,15.1592,15.8391,1571740
...,...,...,...,...,...,...,...
2030248,VCB,2022-10-10,66.5000,67.7000,64.1000,65.1000,1317000
2030249,VCB,2022-10-11,65.3000,66.0000,62.0000,62.0000,1161700
2030250,VCB,2022-10-12,63.0000,65.3000,62.2000,64.0000,1517400
2030251,VCB,2022-10-13,64.1000,66.5000,62.7000,66.5000,1596800


In [23]:
vcb['typical_price'] = vcb[['Close', 'High', 'Low']].mean(axis=1)
vcb['money_flow'] = vcb['Volume'] * vcb['typical_price']

vcb['positive_money_flow'] = np.where(vcb['typical_price'].diff(1) > 0, vcb['typical_price'].shift(1), 0)
vcb['negative_money_flow'] = np.where(vcb['typical_price'].diff(1) < 0, vcb['typical_price'].shift(1), 0)
vcb['neutral_money_flow'] = np.where(vcb['typical_price'].diff(1) == 0, 0, 0)

vcb['money_flow_ratio'] = vcb['positive_money_flow'].rolling(14).mean()/vcb['negative_money_flow'].rolling(14).mean()
vcb['mfi'] = 100 - 100/(1+vcb['money_flow_ratio'])

vcb

,Ticker,Date,Open,High,Low,Close,Volume,typical_price,money_flow,positive_money_flow,negative_money_flow,neutral_money_flow,money_flow_ratio,mfi
2026933,VCB,2009-06-30,16.2420,16.2420,16.2420,16.2452,294070,16.243067,4.776599e+06,0.000000,0.000000,0,NaN,NaN
2026934,VCB,2009-07-01,17.0541,17.0541,16.1067,16.3806,6248390,16.513800,1.031847e+08,16.243067,0.000000,0,NaN,NaN
2026935,VCB,2009-07-02,16.1067,16.2420,15.5653,15.7037,1515670,15.837000,2.400367e+07,0.000000,16.513800,0,NaN,NaN
2026936,VCB,2009-07-03,15.2946,15.4299,15.1592,15.1622,899720,15.250433,1.372112e+07,0.000000,15.837000,0,NaN,NaN
2026937,VCB,2009-07-06,15.1592,15.8360,15.1592,15.8391,1571740,15.611433,2.453711e+07,15.250433,0.000000,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030248,VCB,2022-10-10,66.5000,67.7000,64.1000,65.1000,1317000,65.633333,8.643910e+07,0.000000,67.200000,0,0.163171,14.028153
2030249,VCB,2022-10-11,65.3000,66.0000,62.0000,62.0000,1161700,63.333333,7.357433e+07,0.000000,65.633333,0,0.165705,14.215000
2030250,VCB,2022-10-12,63.0000,65.3000,62.2000,64.0000,1517400,63.833333,9.686070e+07,63.333333,0.000000,0,0.261281,20.715537
2030251,VCB,2022-10-13,64.1000,66.5000,62.7000,66.5000,1596800,65.233333,1.041646e+08,63.833333,0.000000,0,0.378056,27.434014


In [35]:
df['typical_price'] = df[['Close', 'High', 'Low']].mean(axis=1)
df['money_flow'] = df['Volume'] * df['typical_price']
df['positive_money_flow'] = np.where(df.groupby('Ticker')['typical_price'].diff(1) > 0, df.groupby('Ticker')['typical_price'].shift(1), 0)
df['negative_money_flow'] = np.where(df.groupby('Ticker')['typical_price'].diff(1) < 0, df.groupby('Ticker')['typical_price'].shift(1), 0)
df['neutral_money_flow'] = np.where(df.groupby('Ticker')['typical_price'].diff(1) == 0, 0, 0)
df['positive_money_flow_ma_14'] = df.groupby('Ticker')['positive_money_flow'].rolling(14).mean().reset_index(drop=True).to_list()
df['negative_money_flow_ma_14'] = df.groupby('Ticker')['negative_money_flow'].rolling(14).mean().reset_index(drop=True).to_list()

df['money_flow_ratio'] = df['positive_money_flow_ma_14']/df['negative_money_flow_ma_14']
df['mfi'] = 100 - 100/(1+df['money_flow_ratio'])

df

,Ticker,Date,Open,High,Low,Close,Volume,typical_price,money_flow,positive_money_flow,negative_money_flow,neutral_money_flow,positive_money_flow_ma_14,negative_money_flow_ma_14,money_flow_ratio,mfi
0,A32,2018-11-29,22.9056,22.9056,22.9056,22.9076,1000,22.906267,22906.266667,0.000000,0.000000,0,NaN,NaN,NaN,NaN
1,A32,2018-12-03,22.9056,22.9056,22.9056,22.9076,100,22.906267,2290.626667,0.000000,0.000000,0,NaN,NaN,NaN,NaN
2,A32,2018-12-04,20.0424,20.0424,20.0424,20.0441,2000,20.042967,40085.933333,0.000000,22.906267,0,NaN,NaN,NaN,NaN
3,A32,2018-12-21,21.6172,21.6172,21.6172,21.6190,100,21.617800,2161.780000,20.042967,0.000000,0,NaN,NaN,NaN,NaN
4,A32,2019-01-02,22.9056,22.9056,22.9056,22.9076,100,22.906267,2290.626667,21.617800,0.000000,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2274182,YTC,2022-05-12,68.0000,68.0000,68.0000,68.0000,100,68.000000,6800.000000,0.000000,74.800000,0,29.240476,15.538095,1.881857,65.300154
2274183,YTC,2022-06-15,66.0000,66.0000,65.0000,65.0000,200,65.333333,13066.666667,0.000000,68.000000,0,25.883333,20.395238,1.269087,55.929413
2274184,YTC,2022-06-24,65.5000,65.5000,65.5000,65.5000,10,65.500000,655.000000,65.333333,0.000000,0,26.978571,20.395238,1.322788,56.948284
2274185,YTC,2022-09-05,65.5000,65.5000,65.5000,65.5000,200,65.500000,13100.000000,0.000000,0.000000,0,23.050000,20.395238,1.130166,53.055297


In [36]:
df.loc[df['Ticker'] == 'VCB']

,Ticker,Date,Open,High,Low,Close,Volume,typical_price,money_flow,positive_money_flow,negative_money_flow,neutral_money_flow,positive_money_flow_ma_14,negative_money_flow_ma_14,money_flow_ratio,mfi
2026933,VCB,2009-06-30,16.2420,16.2420,16.2420,16.2452,294070,16.243067,4.776599e+06,0.000000,0.000000,0,NaN,NaN,NaN,NaN
2026934,VCB,2009-07-01,17.0541,17.0541,16.1067,16.3806,6248390,16.513800,1.031847e+08,16.243067,0.000000,0,NaN,NaN,NaN,NaN
2026935,VCB,2009-07-02,16.1067,16.2420,15.5653,15.7037,1515670,15.837000,2.400367e+07,0.000000,16.513800,0,NaN,NaN,NaN,NaN
2026936,VCB,2009-07-03,15.2946,15.4299,15.1592,15.1622,899720,15.250433,1.372112e+07,0.000000,15.837000,0,NaN,NaN,NaN,NaN
2026937,VCB,2009-07-06,15.1592,15.8360,15.1592,15.8391,1571740,15.611433,2.453711e+07,15.250433,0.000000,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030248,VCB,2022-10-10,66.5000,67.7000,64.1000,65.1000,1317000,65.633333,8.643910e+07,0.000000,67.200000,0,10.392857,63.692857,0.163171,14.028153
2030249,VCB,2022-10-11,65.3000,66.0000,62.0000,62.0000,1161700,63.333333,7.357433e+07,0.000000,65.633333,0,10.392857,62.719048,0.165705,14.215000
2030250,VCB,2022-10-12,63.0000,65.3000,62.2000,64.0000,1517400,63.833333,9.686070e+07,63.333333,0.000000,0,14.916667,57.090476,0.261281,20.715537
2030251,VCB,2022-10-13,64.1000,66.5000,62.7000,66.5000,1596800,65.233333,1.041646e+08,63.833333,0.000000,0,19.476190,51.516667,0.378056,27.434014


In [9]:
def mfi(panel_data, lookback: int):
    panel_data['typical_price'] = panel_data[['Close', 'High', 'Low']].mean(axis=1)
    panel_data['money_flow'] = panel_data['Volume'] * panel_data['typical_price']
    panel_data['positive_money_flow'] = np.where(panel_data.groupby('Ticker')['typical_price'].diff(1) > 0, panel_data.groupby('Ticker')['typical_price'].shift(1), 0)
    panel_data['negative_money_flow'] = np.where(panel_data.groupby('Ticker')['typical_price'].diff(1) < 0, panel_data.groupby('Ticker')['typical_price'].shift(1), 0)
    panel_data['neutral_money_flow'] = np.where(panel_data.groupby('Ticker')['typical_price'].diff(1) == 0, 0, 0)
    panel_data['positive_money_flow_ma'] = panel_data.groupby('Ticker')['positive_money_flow'].rolling(lookback).mean().reset_index(drop=True).to_list()
    panel_data['negative_money_flow_ma'] = panel_data.groupby('Ticker')['negative_money_flow'].rolling(lookback).mean().reset_index(drop=True).to_list()

    panel_data['money_flow_ratio'] = panel_data['positive_money_flow_ma']/panel_data['negative_money_flow_ma']
    panel_data[f'mfi_{lookback}'] = 100 - 100/(1+panel_data['money_flow_ratio'])
    
    del panel_data['typical_price'] 
    del panel_data['money_flow'] 
    del panel_data['positive_money_flow']
    del panel_data['negative_money_flow']
    del panel_data['neutral_money_flow']
    del panel_data['positive_money_flow_ma'] 
    del panel_data['negative_money_flow_ma']
    del panel_data['money_flow_ratio']

    return panel_data

In [10]:
df = mfi(panel_data=df, lookback=14)
df

,Ticker,Date,Open,High,Low,Close,Volume,mfi_14
0,A32,2018-11-29,22.9056,22.9056,22.9056,22.9076,1000,NaN
1,A32,2018-12-03,22.9056,22.9056,22.9056,22.9076,100,NaN
2,A32,2018-12-04,20.0424,20.0424,20.0424,20.0441,2000,NaN
3,A32,2018-12-21,21.6172,21.6172,21.6172,21.6190,100,NaN
4,A32,2019-01-02,22.9056,22.9056,22.9056,22.9076,100,NaN
...,...,...,...,...,...,...,...,...
2274182,YTC,2022-05-12,68.0000,68.0000,68.0000,68.0000,100,65.300154
2274183,YTC,2022-06-15,66.0000,66.0000,65.0000,65.0000,200,55.929413
2274184,YTC,2022-06-24,65.5000,65.5000,65.5000,65.5000,10,56.948284
2274185,YTC,2022-09-05,65.5000,65.5000,65.5000,65.5000,200,53.055297
